# Sensitivity of Bear and Weber WEAP Models System's Reliability due to changes in input 

#### By Adel M. Abdallah, Utah State University, April 2019

Execute the following cells by pressing `Shift-Enter`, or by pressing the play button <img style='display:inline;padding-bottom:15px' src='play-button.png'> on the toolbar above.



<a name="Import"></a>
# 1. Import python libraries 

In [80]:
# 1. Import python libraries 
### set the notebook mode to embed the figures within the cell
import numpy
import sqlite3
import numpy as np
import pandas as pd
import getpass
from hs_restclient import HydroShare, HydroShareAuthBasic
import os

import plotly
plotly.__version__
import plotly.offline as offline
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
offline.init_notebook_mode(connected=True)
from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import *

init_notebook_mode(connected=True)         # initiate notebook for offline plot

import os
import csv
from collections import OrderedDict
import sqlite3
import pandas as pd
import numpy as np
from IPython.display import display, Image, SVG, Math, YouTubeVideo
import urllib
import calendar

print 'The needed Python libraries have been imported'

The needed Python libraries have been imported



# 3. Download the Bear River WEAP model and connect Jupyter Notebook to WEAP API

Clone or download all this GitHub repo   
https://github.com/WamdamProject/WaMDaM_UseCases  

In your local repo folder, go to the    
    
    C:\Users\Adel\Documents\GitHub\WaMDaM_UseCases/UseCases_files/1Original_Datasets_preperation_files/WEAP/Bear_River_WEAP_Model_2017

Copy this folder **Bear_River_WEAP_Model_2017** and paste it into **WEAP Areas** folder on your local machine. For example, it is at   

    C:\Users\Adel\Documents\WEAP Areas  


<a name="ConnectWEAP"></a>
# 2. Connect to the WEAP API

### You need to have WEAP already installed on your machine

First make sure to have a copy of the Water Evaluation And Planning" system (WEAP) installed on your local machine (Windows). If you don’t have it installed, download and install the WEAP software which allows you to run the Bear River WEAP model and its scenarios for Use Case 5. https://www.weap21.org/. You need to have a WEAP License. See here (https://www.weap21.org/index.asp?action=217). If you're interested to learning about WEAP API, check it out here: http://www.weap21.org/WebHelp/API.htm    


## Install dependency and register WEAP
### 2.1. Install pywin32 extensions which provide access to many of the Windows APIs from Python.
**Choose on option**
* a. Install using an executable basedon your python version. Use version for Python 2.7
https://github.com/mhammond/pywin32/releases 

**OR**   

* b. Install it using Anaconda terminal @ https://anaconda.org/anaconda/pywin32

Type this command in the Anaconda terminal as Administrator  

    conda install -c anaconda pywin32 


**OR**

* c. Install from source code (for advanced users) 
https://github.com/mhammond/pywin32


### 2.2. Register WEAP with Windows 


This use case only works on a local Jupyter Notebook server installed on your machine along with WEAP. So it does not work on the online Notebooks in Step 2.1. You need to install Jupyter Server in Step 2.2 then proceed here.




* **Register WEAP with Windows to allow the WEAP API to be accessed**    
Use Windows "Command Prompt". Right click and then <font color=red>**run as Administrator**</font>, navigate to the WEAP installation directory such as and then hit enter  

```
cd C:\Program Files (x86)\WEAP
```

Then type the following command in the command prompt and hit enter   
```
WEAP /regserver
```


<img src="https://github.com/WamdamProject/WaMDaM-software-ecosystem/blob/master/mkdocs/Edit_MD_Files/QuerySelect/images/RegisterWEAP_CMD.png?raw=true" style="float:center;width:700px;padding:20px"> 
Figure 1: Register WEAP API with windows using the Command Prompt (Run as Administrator)


# Connect to WEAP and Create the sedimentation WEAP Area /Scenariois

<a name="CreateWEAP_Area"></a>
# 3.1 Create a copy of the original WEAP Area to use while keeping the orignial as-as for any later use


<a name="AddScenarios"></a>
### Add a new CacheCountyUrbanWaterUse scenario from the Reference original WEAP Area:  

### You can always use this orignal one and delete any new copies you make afterwards.

In [81]:
# this library is needed to connect to the WEAP API
import win32com.client

# this command will open the WEAP software (if closed) and get the last active model
# you could change the active area to another one inside WEAP or by passing it to the command here
#WEAP.ActiveArea = "BearRiverFeb2017_V10.9"


WEAP=win32com.client.Dispatch("WEAP.WEAPApplication")

# WEAP.Visible = 'FALSE'

#-----------------------------------------------------------------

print WEAP.ActiveArea.Name 
WEAP.ActiveArea = "Bear_River_WEAP_Model_2017_Original"  
print WEAP.ActiveArea.Name 

WEAP.Areas("Bear_River_WEAP_Model_2017_Original").Open
WEAP.ActiveArea = "Bear_River_WEAP_Model_2017_Original"  
print WEAP.ActiveArea.Name
#-----------------------------------------------------------------


print 'Connected to WEAP API and the '+ WEAP.ActiveArea.Name + ' Area'
print '-------------'
if not WEAP.Registered:
    print "Because WEAP is not registered, you cannot use the API"

# get the active WEAP Area (model) to serve data into it 

# ActiveArea=WEAP.ActiveArea.Name 


# get the active WEAP scenario to serve data into it 
print '-------------'

ActiveScenario= WEAP.ActiveScenario.Name
print '\n ActiveScenario= '+ActiveScenario
print '-------------'

WEAP_Area_dir=WEAP.AreasDirectory
print WEAP_Area_dir


print "\n \n You're connected to the WEAP API"


# Delete the Area if it exists and then add it. Start from fresh
Area="Bear_River_WEAP_Model_2017_sedimentation"

if not WEAP.Areas.Exists(Area):
    WEAP.SaveAreaAs(Area)
WEAP.Quit
print " Created the Bear_River_WEAP_Model_2017_sedimentation Area"

WEAP=win32com.client.Dispatch("WEAP.WEAPApplication")

#-----------------------------------------------------------------
print WEAP.ActiveArea.Name 
WEAP.ActiveArea = "Bear_River_WEAP_Model_2017_Original"  
print WEAP.ActiveArea.Name 

WEAP.Areas("Bear_River_WEAP_Model_2017_Original").Open
WEAP.ActiveArea = "Bear_River_WEAP_Model_2017_Original"  
print WEAP.ActiveArea.Name
#-----------------------------------------------------------------


# Delete the Area if it exists and then add it. Start from fresh
Area="Bear_River_WEAP_Model_2017_scenarios"

if not WEAP.Areas.Exists(Area):
    WEAP.SaveAreaAs(Area)
print  'ActiveArea= '+ WEAP.ActiveArea.Name


WEAP.Areas("Bear_River_WEAP_Model_2017_scenarios").Open
WEAP.ActiveArea = "Bear_River_WEAP_Model_2017_scenarios"  
print WEAP.ActiveArea.Name

Scenarios=[]
Scenarios=['Evaporation','Headflow','Demand']

for Scenario in Scenarios:
    if WEAP.Scenarios.Exists(Scenario):
        # delete it
        WEAP.Scenarios(Scenario).Delete(True)
        # add it back as a fresh copy
        WEAP.Scenarios.Add(Scenario,'Reference')
    else:
        WEAP.Scenarios.Add(Scenario,'Reference')
    
WEAP.ActiveArea.Save
WEAP.SaveArea
# Make a copy from the reference (base) scenario
# WEAP.Scenarios.Add('UpdateCacheDemand','Reference')
print '---------------------- \n'
print 'Scenarios added to the Bear_River_WEAP_Model_2017_scenarios WEAP area'  

Bear_River_WEAP_Model_2017_scenarios
Bear_River_WEAP_Model_2017_scenarios
Bear_River_WEAP_Model_2017_scenarios
Connected to WEAP API and the Bear_River_WEAP_Model_2017_scenarios Area
-------------
-------------

 ActiveScenario= Current Accounts
-------------
C:\Users\Adel\Documents\WEAP Areas\

 
 You're connected to the WEAP API
 Created the Bear_River_WEAP_Model_2017_sedimentation Area
Bear_River_WEAP_Model_2017_scenarios
Bear_River_WEAP_Model_2017_scenarios
Bear_River_WEAP_Model_2017_scenarios
ActiveArea= Bear_River_WEAP_Model_2017_scenarios
Bear_River_WEAP_Model_2017_scenarios
---------------------- 

Scenarios added to the Bear_River_WEAP_Model_2017_scenarios WEAP area


# Reliability Results: Weeping Model
Reliability is for the entire study period, so doesn't have a year or time step parameter
Reliability to fully meet demand =number of years of zero shortage divided by total number of years

# 5. Run WEAP
<font color=green>**Please wait, it will take ~1-3 minutes** to finish calcualting the two WEAP Areas with their many scenarios</font>

In [85]:
#-----------------------------------------------------------------
print WEAP.ActiveArea.Name 
WEAP.ActiveArea = "Bear_River_WEAP_Model_2017_scenarios"  
print WEAP.ActiveArea.Name 

WEAP.Areas("Bear_River_WEAP_Model_2017_scenarios").Open
WEAP.ActiveArea = "Bear_River_WEAP_Model_2017_scenarios"  
print WEAP.ActiveArea.Name
#-----------------------------------------------------------------


print 'Please wait 1-3 min for the calculation to finish'
# WEAP.Calculate(2006,10,True)
# WEAP.SaveArea

print '\n \n The calculation has been done and saved'
# print WEAP.CalculationTime

print 'Done'

SelectDemandSites=['Montpelier Irr','Highline Canal','Hyrum Canal','Bird Refuge','Logan Potable']



Result= OrderedDict()
# columns=['ScenarioName','BranchName','Reliability']
# Result=OrderedDict() 

# group:ScenarioName
# Key=BranchName
# value:BranchName
    
for ScenarioName in WEAP.Scenarios:
    print ScenarioName
    WEAP.ActiveScenario=ScenarioName

    #print str(ScenarioName)
    scenarioKey = str(ScenarioName)
    Result[scenarioKey] = OrderedDict()
    
    BranchNames = []
    Reliabilities = []
    
    for Branch in WEAP.Branches:
        if Branch.TypeName=='Demand Site' and Branch.IsNode:
            FullBranchName=Branch.FullName
            BranchName=Branch.Name
            for SelectSite in SelectDemandSites:
                if SelectSite==BranchName:
                    if WEAP.Branch(FullBranchName).Variables.Exists("Reliability"):
                        BranchNames.append(BranchName)
                        Reliabilities.append(WEAP.Branch(FullBranchName).Variables("Reliability").Value)
    
    NewReliabilities = []
    for site in SelectDemandSites:
        index = BranchNames.index(site)
        NewReliabilities.append(Reliabilities[index])
    
#     print BranchNames
    Result[scenarioKey]['BranchName'] = SelectDemandSites
    Result[scenarioKey]['Reliability'] = NewReliabilities

print 'done'
print Result            
            
 

WeberOgdenRiversLab-3_scenarios
Bear_River_WEAP_Model_2017_scenarios
Bear_River_WEAP_Model_2017_scenarios
Please wait 1-3 min for the calculation to finish

 
 The calculation has been done and saved
Done
Current Accounts
Reference
Evaporation
Headflow
Demand
done
OrderedDict([('Current Accounts', OrderedDict([('BranchName', ['Montpelier Irr', 'Highline Canal', 'Hyrum Canal', 'Bird Refuge', 'Logan Potable']), ('Reliability', [50.0, 66.66666666666667, 66.66666666666667, 91.66666666666667, 100.0])])), ('Reference', OrderedDict([('BranchName', ['Montpelier Irr', 'Highline Canal', 'Hyrum Canal', 'Bird Refuge', 'Logan Potable']), ('Reliability', [50.0, 79.8780487804878, 85.77235772357723, 96.34146341463415, 100.0])])), ('Evaporation', OrderedDict([('BranchName', ['Montpelier Irr', 'Highline Canal', 'Hyrum Canal', 'Bird Refuge', 'Logan Potable']), ('Reliability', [50.0, 79.26829268292683, 85.77235772357723, 96.34146341463415, 100.0])])), ('Headflow', OrderedDict([('BranchName', ['Montpelier 

# Get the Sedeimtation Scenarios

In [78]:
WEAP=win32com.client.Dispatch("WEAP.WEAPApplication")

#-----------------------------------------------------------------
print WEAP.ActiveArea.Name 
WEAP.ActiveArea = "Bear_River_WEAP_Model_2017_sedimentation"  
print WEAP.ActiveArea.Name 

WEAP.Areas("Bear_River_WEAP_Model_2017_sedimentation").Open
WEAP.ActiveArea = "Bear_River_WEAP_Model_2017_sedimentation"  
print WEAP.ActiveArea.Name
#-----------------------------------------------------------------


print 'Please wait 1-3 min for the calculation to finish'
# WEAP.Calculate(2006,10,True)
WEAP.SaveArea

print '\n \n The calculation has been done and saved'
# print WEAP.CalculationTime

print 'Done'

# WEAP.Visible = 'FALSE'

SelectDemandSites=['Montpelier Irr','Highline Canal','Hyrum Canal','Bird Refuge','Logan Potable']



Result2= OrderedDict()
    
for ScenarioName in WEAP.Scenarios:
    print ScenarioName

#     if ScenarioName=='Reference':
    WEAP.ActiveScenario=ScenarioName

    #print str(ScenarioName)
    scenarioKey = str(ScenarioName)
    Result2[scenarioKey] = OrderedDict()

    BranchNames = []
    Reliabilities = []

    for Branch in WEAP.Branches:
        if Branch.TypeName=='Demand Site' and Branch.IsNode:
            FullBranchName=Branch.FullName
            BranchName=Branch.Name
            for SelectSite in SelectDemandSites:
                if SelectSite==BranchName:
                    if WEAP.Branch(FullBranchName).Variables.Exists("Reliability"):
                        BranchNames.append(BranchName)
                        Reliabilities.append(WEAP.Branch(FullBranchName).Variables("Reliability").Value)
#     NewBranchNames = []
    NewReliabilities = []
    for site in SelectDemandSites:
        index = BranchNames.index(site)
        NewReliabilities.append(Reliabilities[index])
    Result2[scenarioKey]['BranchName'] = SelectDemandSites
    Result2[scenarioKey]['Reliability'] = NewReliabilities

print 'done'
print Result2            

Bear_River_WEAP_Model_2017_scenarios
Bear_River_WEAP_Model_2017_sedimentation
Bear_River_WEAP_Model_2017_sedimentation
Please wait 1-3 min for the calculation to finish

 
 The calculation has been done and saved
Done
Current Accounts
Reference
done
OrderedDict([('Current Accounts', OrderedDict([('BranchName', ['Montpelier Irr', 'Highline Canal', 'Hyrum Canal', 'Bird Refuge', 'Logan Potable']), ('Reliability', [50.0, 66.66666666666667, 66.66666666666667, 91.66666666666667, 100.0])])), ('Reference', OrderedDict([('BranchName', ['Montpelier Irr', 'Highline Canal', 'Hyrum Canal', 'Bird Refuge', 'Logan Potable']), ('Reliability', [50.0, 77.64227642276423, 84.95934959349593, 96.34146341463415, 100.0])]))])


# Get the Weber scenarios 

In [83]:
# this library is needed to connect to the WEAP API
import win32com.client

# this command will open the WEAP software (if closed) and get the last active model
# you could change the active area to another one inside WEAP or by passing it to the command here
#WEAP.ActiveArea = "BearRiverFeb2017_V10.9"


WEAP=win32com.client.Dispatch("WEAP.WEAPApplication")

# WEAP.Visible = 'FALSE'

#-----------------------------------------------------------------

print WEAP.ActiveArea.Name 
WEAP.ActiveArea = "WeberOgdenRiversLab-3_Original"  
print WEAP.ActiveArea.Name 

WEAP.Areas("WeberOgdenRiversLab-3_Original").Open
WEAP.ActiveArea = "WeberOgdenRiversLab-3_Original"  
print WEAP.ActiveArea.Name
#-----------------------------------------------------------------


print 'Connected to WEAP API and the '+ WEAP.ActiveArea.Name + ' Area'
print '-------------'
if not WEAP.Registered:
    print "Because WEAP is not registered, you cannot use the API"

# get the active WEAP Area (model) to serve data into it 

# ActiveArea=WEAP.ActiveArea.Name 


# get the active WEAP scenario to serve data into it 
print '-------------'

ActiveScenario= WEAP.ActiveScenario.Name
print '\n ActiveScenario= '+ActiveScenario
print '-------------'

WEAP_Area_dir=WEAP.AreasDirectory
print WEAP_Area_dir


print "\n \n You're connected to the WEAP API"


# Delete the Area if it exists and then add it. Start from fresh
Area="WeberOgdenRiversLab-3_sedimentation"

if not WEAP.Areas.Exists(Area):
    WEAP.SaveAreaAs(Area)
WEAP.Quit
print " Created the WeberOgdenRiversLab-3_sedimentation Area"

WEAP=win32com.client.Dispatch("WEAP.WEAPApplication")

#-----------------------------------------------------------------
print WEAP.ActiveArea.Name 
WEAP.ActiveArea = "WeberOgdenRiversLab-3_Original"  
print WEAP.ActiveArea.Name 

WEAP.Areas("WeberOgdenRiversLab-3_Original").Open
WEAP.ActiveArea = "WeberOgdenRiversLab-3_Original"  
print WEAP.ActiveArea.Name
#-----------------------------------------------------------------


# Delete the Area if it exists and then add it. Start from fresh
Area="WeberOgdenRiversLab-3_scenarios"

if not WEAP.Areas.Exists(Area):
    WEAP.SaveAreaAs(Area)
print  'ActiveArea= '+ WEAP.ActiveArea.Name


WEAP.Areas("WeberOgdenRiversLab-3_scenarios").Open
WEAP.ActiveArea = "WeberOgdenRiversLab-3_scenarios"  
print WEAP.ActiveArea.Name

Scenarios=[]
Scenarios=['Evaporation','Headflow','Demand']

for Scenario in Scenarios:
    if WEAP.Scenarios.Exists(Scenario):
        # delete it
        WEAP.Scenarios(Scenario).Delete(True)
        # add it back as a fresh copy
        WEAP.Scenarios.Add(Scenario,'Reference')
    else:
        WEAP.Scenarios.Add(Scenario,'Reference')
    
WEAP.ActiveArea.Save
WEAP.SaveArea
# Make a copy from the reference (base) scenario
# WEAP.Scenarios.Add('UpdateCacheDemand','Reference')
print '---------------------- \n'
print 'Scenarios added to the WeberOgdenRiversLab-3_scenarios WEAP area'  

Bear_River_WEAP_Model_2017_scenarios
Bear_River_WEAP_Model_2017_scenarios


AttributeError: 'NoneType' object has no attribute 'Open'

# get the scenariois reliability 

In [84]:
import win32com.client

WEAP=win32com.client.Dispatch("WEAP.WEAPApplication")
#-----------------------------------------------------------------
print WEAP.ActiveArea.Name 
WEAP.ActiveArea = "WeberOgdenRiversLab-3_scenarios"  
print WEAP.ActiveArea.Name 

WEAP.Areas("WeberOgdenRiversLab-3_scenarios").Open
WEAP.ActiveArea = "WeberOgdenRiversLab-3_scenarios"  
print WEAP.ActiveArea.Name
#-----------------------------------------------------------------


print 'Please wait 1-3 min for the calculation to finish'
# WEAP.Calculate(2006,10,True)
# WEAP.SaveArea

print '\n \n The calculation has been done and saved'
# print WEAP.CalculationTime

print 'Done'

SelectDemandSites=['Weber Basin Proj. Ogd Valley','Wanship to Echo']



Result3= OrderedDict()
# columns=['ScenarioName','BranchName','Reliability']
# Result=OrderedDict() 

# group:ScenarioName
# Key=BranchName
# value:BranchName
    
for ScenarioName in WEAP.Scenarios:
    print ScenarioName
    WEAP.ActiveScenario=ScenarioName

    #print str(ScenarioName)
    scenarioKey = str(ScenarioName)
    Result3[scenarioKey] = OrderedDict()
    
    BranchNames = []
    Reliabilities = []
    
    for Branch in WEAP.Branches:
        if Branch.TypeName=='Demand Site' and Branch.IsNode:
            FullBranchName=Branch.FullName
            BranchName=Branch.Name
            for SelectSite in SelectDemandSites:
                if SelectSite==BranchName:
                    if WEAP.Branch(FullBranchName).Variables.Exists("Reliability"):
                        #Result['ScenarioName'] = ScenarioName
                        BranchNames.append(BranchName)
                        Reliabilities.append(WEAP.Branch(FullBranchName).Variables("Reliability").Value)
    
    NewReliabilities = []
    for site in SelectDemandSites:
        index = BranchNames.index(site)
        NewReliabilities.append(Reliabilities[index])
    
    Result3[scenarioKey]['BranchName'] = SelectDemandSites
    Result3[scenarioKey]['Reliability'] = NewReliabilities

print 'done'
            
print Result3            
 

Bear_River_WEAP_Model_2017_scenarios
WeberOgdenRiversLab-3_scenarios
WeberOgdenRiversLab-3_scenarios
Please wait 1-3 min for the calculation to finish

 
 The calculation has been done and saved
Done
Current Accounts
Reference
Evaporation
Headflow
Demand
done
OrderedDict([('Current Accounts', OrderedDict([('BranchName', ['Weber Basin Proj. Ogd Valley', 'Wanship to Echo']), ('Reliability', [100.0, 100.0])])), ('Reference', OrderedDict([('BranchName', ['Weber Basin Proj. Ogd Valley', 'Wanship to Echo']), ('Reliability', [97.51461988304094, 100.0])])), ('Evaporation', OrderedDict([('BranchName', ['Weber Basin Proj. Ogd Valley', 'Wanship to Echo']), ('Reliability', [97.51461988304094, 100.0])])), ('Headflow', OrderedDict([('BranchName', ['Weber Basin Proj. Ogd Valley', 'Wanship to Echo']), ('Reliability', [96.34502923976608, 100.0])])), ('Demand', OrderedDict([('BranchName', ['Weber Basin Proj. Ogd Valley', 'Wanship to Echo']), ('Reliability', [98.83040935672514, 100.0])]))])


# Get the sedemionation

In [86]:
WEAP=win32com.client.Dispatch("WEAP.WEAPApplication")

#-----------------------------------------------------------------
print WEAP.ActiveArea.Name 
WEAP.ActiveArea = "WeberOgdenRiversLab-3_sedimentation"  
print WEAP.ActiveArea.Name 

WEAP.Areas("WeberOgdenRiversLab-3_sedimentation").Open
WEAP.ActiveArea = "WeberOgdenRiversLab-3_sedimentation"  
print WEAP.ActiveArea.Name
#-----------------------------------------------------------------


print 'Please wait 1-3 min for the calculation to finish'
# WEAP.Calculate(2006,10,True)
WEAP.SaveArea

print '\n \n The calculation has been done and saved'
# print WEAP.CalculationTime

print 'Done'

# WEAP.Visible = 'FALSE'

SelectDemandSites=['Weber Basin Proj. Ogd Valley','Wanship to Echo']



Result4= OrderedDict()
    
for ScenarioName in WEAP.Scenarios:
    print ScenarioName

#     if ScenarioName=='Reference':
    WEAP.ActiveScenario=ScenarioName

    #print str(ScenarioName)
    scenarioKey = str(ScenarioName)
    Result4[scenarioKey] = OrderedDict()

    BranchNames = []
    Reliabilities = []

    for Branch in WEAP.Branches:
        if Branch.TypeName=='Demand Site' and Branch.IsNode:
            FullBranchName=Branch.FullName
            BranchName=Branch.Name
            for SelectSite in SelectDemandSites:
                if SelectSite==BranchName:
                    if WEAP.Branch(FullBranchName).Variables.Exists("Reliability"):
                        #Result['ScenarioName'] = ScenarioName
                        BranchNames.append(BranchName)
                        Reliabilities.append(WEAP.Branch(FullBranchName).Variables("Reliability").Value)
    
        
    NewReliabilities = []
    for site in SelectDemandSites:
        index = BranchNames.index(site)
        NewReliabilities.append(Reliabilities[index])
    
    
    Result4[scenarioKey]['BranchName'] = SelectDemandSites
    Result4[scenarioKey]['Reliability'] = NewReliabilities

print 'done'
print Result4        

Bear_River_WEAP_Model_2017_scenarios
WeberOgdenRiversLab-3_sedimentation
WeberOgdenRiversLab-3_sedimentation
Please wait 1-3 min for the calculation to finish

 
 The calculation has been done and saved
Done
Current Accounts
Reference
done
OrderedDict([('Current Accounts', OrderedDict([('BranchName', ['Weber Basin Proj. Ogd Valley', 'Wanship to Echo']), ('Reliability', [100.0, 100.0])])), ('Reference', OrderedDict([('BranchName', ['Weber Basin Proj. Ogd Valley', 'Wanship to Echo']), ('Reliability', [96.6374269005848, 100.0])]))])


In [96]:
print Result['Reference']['BranchName'],
print Result['Reference']['Reliability'],

['Montpelier Irr', 'Highline Canal', 'Hyrum Canal', 'Bird Refuge', 'Logan Potable'] [50.0, 79.8780487804878, 85.77235772357723, 96.34146341463415, 100.0]


# Plot reliability

In [98]:
scenario1 = go.Scatter(
    x=Result['Demand']['BranchName'],
    y=Result['Demand']['Reliability'],
    name = 'Reduce demand by 10%',   
    mode = 'lines+markers',

    marker = dict(
        color = '#290AD8'
))


scenario2 = go.Scatter(
    x=Result['Reference']['BranchName'],
    y=Result['Reference']['Reliability'],
    name = 'Base Case', 
        mode = 'lines+markers',

    marker = dict(
        color = '#000000',
    
))

scenario3 = go.Scatter(
    x=Result['Evaporation']['BranchName'],
    y=Result['Evaporation']['Reliability'],
    name = 'Increase evaporation by 10%', 
    mode = 'lines+markers',

    marker = dict(
        color = '#3FA0FF'
))



scenario4 = go.Scatter(
    x=Result2['Reference']['BranchName'],
    y=Result2['Reference']['Reliability'],
    name = 'Reduce reservoir capacities by 10%',   
    mode = 'lines+markers',

    marker = dict(
        color = '#72D9FF'
))



scenario5 = go.Scatter(
    x=Result['Headflow']['BranchName'],
    y=Result['Headflow']['Reliability'],
    name = 'Reduce headflows by 10%',   
    mode = 'lines+markers',

    marker = dict(
        color = '#AAF7FF'
))


#$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$

scenario11 = go.Scatter(
    x=Result3['Demand']['BranchName'],
    y=Result3['Demand']['Reliability'],
    name = 'Reduce demand by 10%',   
    mode = 'lines+markers',
    showlegend=False,

    marker = dict(
        color = '#290AD8'
))


scenario12 = go.Scatter(
    x=Result3['Reference']['BranchName'],
    y=Result3['Reference']['Reliability'],
    name = 'Base Case', 
        mode = 'lines+markers',
    showlegend=False,

    marker = dict(
        color = '#000000',
    
))

scenario13 = go.Scatter(
    x=Result3['Evaporation']['BranchName'],
    y=Result3['Evaporation']['Reliability'],
    name = 'Increase evaporation by 10%', 
    mode = 'lines+markers',
    showlegend=False,

    marker = dict(
        color = '#3FA0FF'
))



scenario14 = go.Scatter(
    x=Result4['Reference']['BranchName'],
    y=Result4['Reference']['Reliability'],
    name = 'Reduce reservoir capacities by 10%',   
    mode = 'lines+markers',
    showlegend=False,

    marker = dict(
        color = '#72D9FF'
))



scenario15 = go.Scatter(
    x=Result3['Headflow']['BranchName'],
    y=Result3['Headflow']['Reliability'],
    name = 'Reduce headflows by 10%',   
    mode = 'lines+markers',
    showlegend=False,
    marker = dict(
        color = '#AAF7FF'
))




layout = dict(
    #title = "Use Case 3.3",
    yaxis = dict(
        title = "Demand reliability (%)",
        tickformat= ',',
        showline=True,
        tick0=40,

        dtick='10',
        ticks='outside',
        range = [40, 105],
        ticklen=10,
        tickcolor='#000',
        gridwidth=1,
        showgrid=True,

                ),
    
    xaxis = dict(
#         title = "Updated input parameters in the <br>Bear_River_WEAP_Model_2017",
#         showline=True,
        ticks='inside',
        tickfont=dict(size=22),
            tickcolor='#000',
        gridwidth=1,
        showgrid=True,

        ticklen=10
                    ),
        legend=dict(
        x=1,y=.56,
          bordercolor='#00000f',
            borderwidth=2
        
               ),
    width=1100,
    height=700,
    #paper_bgcolor='rgb(233,233,233)',
    #plot_bgcolor='rgb(233,233,233)',
    margin=go.Margin(l=130,b=200),
    font=dict(size=25,family='arial',color='#00000f'),

    showlegend=True,
    shapes=[{'type': 'line', 
                                           'x0': 0, 'x1': 1, 'xref': 'paper',
                                           'y0': 40, 'y1': 40, 'yref': 'y'}]
                )


data = [scenario1, scenario2,scenario3,scenario4,scenario5,scenario11, scenario12,scenario13,scenario14,scenario15]


# create a figure object
fig = dict(data=data, layout=layout)
#py.iplot(fig, filename = "2.3Identify_SeasonalValues") 


## it can be run from the local machine on Pycharm like this like below
## It would also work here offline but in a seperate window  
offline.iplot(fig,filename = 'jupyter/UnmentDemand@BirdRefuge' )       

print "Figure x is replicated!!"

Figure x is replicated!!


# Bar

In [111]:
scenario1 = go.Bar(
    x=Result['Demand']['BranchName'],
    y=Result['Demand']['Reliability'],
    name = 'Reduce demand by 10%',   

    marker = dict(
        color = '#290AD8'
))


scenario2 = go.Bar(
    x=Result['Reference']['BranchName'],
    y=Result['Reference']['Reliability'],
    name = 'Base Case', 
    marker = dict(
        color = '#000000',
    
))

scenario3 = go.Bar(
    x=Result['Evaporation']['BranchName'],
    y=Result['Evaporation']['Reliability'],
    name = 'Increase evaporation by 10%', 

    marker = dict(
        color = '#3FA0FF'
))



scenario4 = go.Bar(
    x=Result2['Reference']['BranchName'],
    y=Result2['Reference']['Reliability'],
    name = 'Reduce reservoir capacities by 10%',   

    marker = dict(
        color = '#72D9FF'
))



scenario5 = go.Bar(
    x=Result['Headflow']['BranchName'],
    y=Result['Headflow']['Reliability'],
    name = 'Reduce headflows by 10%',   

    marker = dict(
        color = '#AAF7FF'
))


#$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$

scenario11 = go.Bar(
    x=Result3['Demand']['BranchName'],
    y=Result3['Demand']['Reliability'],
    name = 'Reduce demand by 10%',   
#     showlegend=False,

    marker = dict(
        color = '#A50021'
))


scenario12 = go.Bar(
    x=Result3['Reference']['BranchName'],
    y=Result3['Reference']['Reliability'],
    name = 'Base Case', 
#     showlegend=False,

    marker = dict(
        color = '#000000',
    
))

scenario13 = go.Bar(
    x=Result3['Evaporation']['BranchName'],
    y=Result3['Evaporation']['Reliability'],
    name = 'Increase evaporation by 10%', 
#     showlegend=False,

    marker = dict(
        color = '#D82632'
))



scenario14 = go.Bar(
    x=Result4['Reference']['BranchName'],
    y=Result4['Reference']['Reliability'],
    name = 'Reduce reservoir capacities by 10%',   
#     showlegend=False,

    marker = dict(
        color = '#F76D5E'
))



scenario15 = go.Bar(
    x=Result3['Headflow']['BranchName'],
    y=Result3['Headflow']['Reliability'],
    name = 'Reduce headflows by 10%',   
#     showlegend=False,
    marker = dict(
        color = '#FFAD72'
))




layout = dict(
    #title = "Use Case 3.3",
    yaxis = dict(
        title = "Demand reliability (%)",
        tickformat= ',',
        showline=True,
        tick0=40,

        dtick='10',
        ticks='outside',
        range = [40, 105],
        ticklen=10,
        tickcolor='#000',
        gridwidth=1,
        showgrid=True,

                ),
    
    xaxis = dict(
#         title = "Updated input parameters in the <br>Bear_River_WEAP_Model_2017",
#         showline=True,
        ticks='inside',
        tickfont=dict(size=22),
            tickcolor='#000',
        gridwidth=1,
        showgrid=True,

        ticklen=10
                    ),
        legend=dict(
        x=1,y=.56,
          bordercolor='#00000f',
            borderwidth=2
        
               ),
    width=1100,
    height=700,
    #paper_bgcolor='rgb(233,233,233)',
    #plot_bgcolor='rgb(233,233,233)',
    margin=go.Margin(l=130,b=200),
    font=dict(size=25,family='arial',color='#00000f'),

    showlegend=True,
    shapes=[{'type': 'line', 
                                           'x0': 0, 'x1': 1, 'xref': 'paper',
                                           'y0': 40, 'y1': 40, 'yref': 'y'}]
                )


data = [scenario1, scenario2,scenario3,scenario4,scenario5,scenario11, scenario12,scenario13,scenario14,scenario15]


# create a figure object
fig = dict(data=data, layout=layout)
#py.iplot(fig, filename = "2.3Identify_SeasonalValues") 


## it can be run from the local machine on Pycharm like this like below
## It would also work here offline but in a seperate window  
offline.iplot(fig,filename = 'jupyter/UnmentDemand@BirdRefuge' )       

print "Figure x is replicated!!"

Figure x is replicated!!


In [93]:
scenario1 = go.Scatter(
    x=Result['Demand']['BranchName'],
    y=Result['Demand']['Reliability'],
    name = 'Reduce demand by 10%',   
    mode = 'lines+markers',

    marker = dict(
        color = '#290AD8'
))


scenario2 = go.Scatter(
    x=Result['Reference']['BranchName'],
    y=Result['Reference']['Reliability'],
    name = 'Base Case', 
        mode = 'lines+markers',

    marker = dict(
        color = '#000000',
))

scenario3 = go.Scatter(
    x=Result['Evaporation']['BranchName'],
    y=Result['Evaporation']['Reliability'],
    name = 'Increase evaporation by 10%', 
    mode = 'lines+markers',

    marker = dict(
        color = '#3FA0FF'
))



scenario4 = go.Scatter(
    x=Result2['Reference']['BranchName'],
    y=Result2['Reference']['Reliability'],
    name = 'Reduce reservoir capacities by 10%',   
    mode = 'lines+markers',

    marker = dict(
        color = '#72D9FF'
))



scenario5 = go.Scatter(
    x=Result['Headflow']['BranchName'],
    y=Result['Headflow']['Reliability'],
    name = 'Reduce headflows by 10%',   
    mode = 'lines+markers',

    marker = dict(
        color = '#AAF7FF'
))


#$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$

scenario11 = go.Scatter(
    x=Result3['Demand']['BranchName'],
    y=Result3['Demand']['Reliability'],
    name = 'Reduce demand by 10%',   
    mode = 'lines+markers',
    showlegend=False,

    marker = dict(
        color = '#290AD8'
))


scenario12 = go.Scatter(
    x=Result3['Reference']['BranchName'],
    y=Result3['Reference']['Reliability'],
    name = 'Base Case', 
        mode = 'lines+markers',
    showlegend=False,

    marker = dict(
        color = '#000000',
    
))

scenario13 = go.Scatter(
    x=Result3['Evaporation']['BranchName'],
    y=Result3['Evaporation']['Reliability'],
    name = 'Increase evaporation by 10%', 
    mode = 'lines+markers',
    showlegend=False,

    marker = dict(
        color = '#3FA0FF'
))



scenario14 = go.Scatter(
    x=Result4['Reference']['BranchName'],
    y=Result4['Reference']['Reliability'],
    name = 'Reduce reservoir capacities by 10%',   
    mode = 'lines+markers',
    showlegend=False,

    marker = dict(
        color = '#72D9FF'
))



scenario15 = go.Scatter(
    x=Result3['Headflow']['BranchName'],
    y=Result3['Headflow']['Reliability'],
    name = 'Reduce headflows by 10%',   
    mode = 'lines+markers',
    showlegend=False,
    marker = dict(
        color = '#AAF7FF'
))




layout = dict(
    #title = "Use Case 3.3",
    yaxis = dict(
        title = "Demand reliability (%)",
        tickformat= ',',
        showline=True,
        tick0=40,

        dtick='10',
        ticks='outside',
        range = [40, 105],
        ticklen=10,
        tickcolor='#000',
        gridwidth=1,
        showgrid=True,

                ),
    
    xaxis = dict(
#         title = "Updated input parameters in the <br>Bear_River_WEAP_Model_2017",
#         showline=True,
        ticks='inside',
        tickfont=dict(size=22),
            tickcolor='#000',
        gridwidth=1,
        showgrid=True,

        ticklen=10
                    ),
        legend=dict(
        x=1,y=.56,
          bordercolor='#00000f',
            borderwidth=2
        
               ),
    width=1100,
    height=700,
    #paper_bgcolor='rgb(233,233,233)',
    #plot_bgcolor='rgb(233,233,233)',
    margin=go.Margin(l=130,b=200),
    font=dict(size=25,family='arial',color='#00000f'),

    showlegend=True,
    shapes=[{'type': 'line', 
                                           'x0': 0, 'x1': 1, 'xref': 'paper',
                                           'y0': 40, 'y1': 40, 'yref': 'y'}]
                )


data = [scenario1, scenario2,scenario3,scenario4,scenario5,scenario11, scenario12,scenario13,scenario14,scenario15]


# create a figure object
fig = dict(data=data, layout=layout)
#py.iplot(fig, filename = "2.3Identify_SeasonalValues") 


## it can be run from the local machine on Pycharm like this like below
## It would also work here offline but in a seperate window  
offline.iplot(fig,filename = 'jupyter/UnmentDemand@BirdRefuge' )       

print "Figure x is replicated!!"

Figure x is replicated!!


<a name="Close"></a>
# 7. Close WEAP API connection

In [ ]:
# 9. Close the WEAP API connection


print 'connection disconnected'

# Uncomment 
WEAP.SaveArea



# this command will close WEAP
WEAP.Quit

print 'Connection with WEAP API is disconnected'

# The End :) Congratulations!